In [1]:
%matplotlib nbagg
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from scipy.spatial.distance import cdist
from scipy.cluster.vq import vq
from sklearn import preprocessing
from sklearn import decomposition

In [2]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.datasets import load_files

container_path = 'C:/Users/Subhankari/Desktop/desktop_as_on_13th_april/text books/ML/CML_Assignment_2_sm6202/sklearn_tut_workspace/data/movie_reviews/txt_sentoken'

reviews = load_files(container_path, description=None, categories=None, load_content=True, shuffle=True, encoding=None, decode_error='strict', random_state=0)
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(reviews.data,
                                                    reviews.target)

In [3]:
print len(reviews.data)
print len(reviews.filenames)
print reviews.target[:10]
for t in reviews.target[:10]:
    print(reviews.target_names[t])

2000
2000
[0 1 1 0 1 1 1 1 1 0]
neg
pos
pos
neg
pos
pos
pos
pos
pos
neg


In [4]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
X_train_counts.shape

(1500, 35424)

In [5]:
count_vect.vocabulary_.get(u'algorithm')
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
X_train_tf.shape
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(1500, 35424)

In [6]:
from sklearn.neighbors import KNeighborsClassifier

#from sklearn.naive_bayes import MultinomialNB
clf = KNeighborsClassifier().fit(X_train_tfidf, y_train)
docs_new =['God is love', 'OpenGL on the GPU is fast']
X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

predicted = clf.predict(X_new_tfidf)

for doc, category in zip(docs_new, predicted):
    print('%r => %s' % (doc, reviews.target_names[category]))

'God is love' => pos
'OpenGL on the GPU is fast' => neg


In [7]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', KNeighborsClassifier()),])

In [8]:
text_clf = text_clf.fit(X_train, y_train)

In [9]:
import numpy as np
#twenty_test = fetch_20newsgroups(subset='test',
#                                 categories=categories, shuffle=True, random_state=42)
docs_test = X_test
predicted = text_clf.predict(docs_test)
np.mean(predicted == y_test) 

0.57799999999999996

In [10]:
from sklearn import metrics
print(metrics.classification_report(y_test, predicted,
                                    target_names=reviews.target_names))

             precision    recall  f1-score   support

        neg       0.83      0.20      0.32       250
        pos       0.54      0.96      0.69       250

avg / total       0.69      0.58      0.51       500



In [11]:
metrics.confusion_matrix(y_test, predicted)

array([[ 49, 201],
       [ 10, 240]])

In [ ]:
from sklearn.grid_search import GridSearchCV
parameters = {'vect__ngram_range': [(1, 1), (1, 2)],
               'tfidf__use_idf': (True, False),
               'clf__alpha': (1e-2, 1e-3),
               }
gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1)

In [ ]:
gs_clf = gs_clf.fit(X_train[:400], y_train[:400])
reviews.target_names[gs_clf.predict(['God is love'])]

In [ ]:
best_parameters, score, _ = max(gs_clf.grid_scores_, key=lambda x: x[1])
for param_name in sorted(parameters.keys()):
    print("%s: %r" % (param_name, best_parameters[param_name]))